In [1]:
import argparse
import gym
import numpy as np
import torch
from sac import SAC
import robel
from updated_env import DKittyWalkRandom, DKittyOrientRandomDynamics

In [3]:
env = DKittyOrientRandomDynamics([1]*8) #DKittyWalkRandom()#DKittyOrientRandomDynamics([2.]*8) #gym.make("DKittyOrientRandomDynamics-v0")
env.seed(42)
env.action_space.seed(42)
env._max_episode_steps = 160 ####### TO CHECK ITs consistent with PEARL (Arsen)
torch.manual_seed(42)
np.random.seed(42)
env.reset()

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -5.68181401e-01,  8.13372966e-04, -8.18354146e-04,
        3.08128714e-02,  1.85494567e-03, -2.24033451e-02,  1.64054022e-03,
        1.88618944e-02,  2.47019682e-02,  5.02299015e-03,  1.81845340e-02,
        1.90357339e-02,  2.04830845e-02,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        9.04033404e-02,  5.25536381e-02, -3.25606767e-02, -1.48397087e-01,
       -1.17810695e-01, -1.07837316e-01,  2.66294032e-02, -8.99507324e-02,
        1.54115066e-01,  8.63085207e-02,  1.32161809e-01,  2.22097496e-02,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        1.00000000e+00,  5.38100077e-01,  8.42880957e-01,  8.19425621e-01,
       -5.73185525e-01])

In [4]:
env.render()

Creating window glfw


In [3]:
env.coef

1

In [3]:
class Args:
    def __init__(self):
        self.policy = "Gaussian"
        self.eval = True
        self.gamma = 0.99
        self.tau = 0.005
        self.lr = 0.0003
        self.alpha = 0.2
        self.automatic_entropy_tuning = False
        self.batch_size = 256
        self.num_steps = 1000001
        self.hidden_size = 256
        self.update_per_step = 1
        self.start_steps = 10000
        self.target_update_interval = 1
        self.replay_size = 1000000
        self.cuda = True
args = Args()

In [4]:
agent = SAC(env.observation_space.shape[0], env.action_space, args)

In [5]:
agent.load_model("./models/sac_actor_HalfCheetah-v2_",

                 "./models/sac_critic_HalfCheetah-v2_")

Loading models from ./models/sac_actor_HalfCheetah-v2_ and ./models/sac_critic_HalfCheetah-v2_


RuntimeError: Error(s) in loading state_dict for GaussianPolicy:
	size mismatch for linear1.weight: copying a param with shape torch.Size([256, 52]) from checkpoint, the shape in current model is torch.Size([256, 53]).

In [ ]:
from tqdm import trange

In [ ]:
import numpy as np
np.cos(np.pi/180*25.)

In [ ]:
avg_reward = 0.
avg_success = 0.
episodes = 10
max_test_r = -np.inf
for _  in range(episodes):
    state = env.reset()
    env.render()
    episode_reward = 0
    episode_success = 0
    done = False
    for i in trange(80):
        action = agent.select_action(state, evaluate=True)
        next_state, reward, done, info = env.step(action)
        episode_reward += reward
        state = next_state
        if info['score/success']:
            episode_success = 1
    avg_success += episode_success       
    avg_reward += episode_reward
avg_reward /= episodes
avg_success /= episodes

print("----------------------------------------")
print("Test Episodes: {}, Avg. Success: {}".format(episodes, round(avg_success, 2)))
print("----------------------------------------")
print("Test Episodes: {}, Avg. Reward: {}".format(episodes, round(avg_reward, 2)))
print("----------------------------------------")
env.close()